In [1]:
from keras.preprocessing import sequence
import tensorflow as tf
import keras
import os
import numpy as np

2022-08-03 21:31:03.034041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 21:31:03.034077: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txxt', 'http://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# if you want to use your own file
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

text= open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# encoding

vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

print("Text:", text[:13])
print("Encoding: ", text_to_int(text[:13]))

Text: First Citizen
Encoding:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [5]:
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [6]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

In [7]:
# Creating Training Examples
# feed input sequence, output is original sequence shifted 1 letter to the right
# basically predicts the next char

seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

2022-08-03 21:31:20.403895: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-03 21:31:20.403952: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-08-03 21:31:20.413989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # of unique chars
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# buffer size to shuffle the datatset
BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                 batch_input_shape=[batch_size, None]), # none = don't know lenght of sequence
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


## Creating a loss function

In [12]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch) # ask our model for a prediction on our first batch of training data
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape


(64, 100, 65) # (batch_size, sequence_length, vocab_size)


2022-08-03 21:31:23.065461: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2022-08-03 21:31:23.076598: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.


In [13]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 9.5528122e-03 -1.5443719e-03 -4.7377208e-03 ... -5.0634122e-04
   -5.0597952e-04 -6.6094217e-03]
  [ 8.6294012e-03  1.4757626e-03  9.6377230e-04 ... -5.4431311e-04
    4.2970935e-03 -5.3331284e-03]
  [ 1.1695606e-02  6.0755471e-03 -3.3139391e-03 ...  4.1671931e-03
    1.8944028e-03 -7.1819536e-03]
  ...
  [ 5.6431200e-03 -3.7155938e-03  2.1505128e-03 ... -3.6936773e-03
    3.7387027e-03 -4.3226578e-03]
  [ 1.3665174e-02 -4.4317832e-03 -2.9737600e-03 ... -3.7974138e-03
    2.0989524e-03 -1.0513639e-02]
  [ 9.7278384e-03 -3.4630508e-03 -1.9981777e-03 ... -6.6221622e-03
    3.1717923e-03 -1.2708948e-02]]

 [[ 3.7754264e-03  1.3086944e-03  5.8634910e-03 ...  5.7026162e-04
    4.2824363e-03 -4.7717905e-03]
  [ 3.5868571e-03 -2.3741687e-03  1.5797524e-03 ... -3.9814780e-03
    3.5142768e-03 -3.0483003e-04]
  [ 7.3570805e-03  4.2919698e-03 -3.0486714e-03 ...  2.0886546e-03
    2.1278777e-03 -3.0744071e-03]
  ...
  [ 8.9818817e-03 -6.7843194e-03 -1.7906090e-02 ... -6.3822898e

In [14]:
# one prediction
# 2d array of length 100

pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 0.00955281 -0.00154437 -0.00473772 ... -0.00050634 -0.00050598
  -0.00660942]
 [ 0.0086294   0.00147576  0.00096377 ... -0.00054431  0.00429709
  -0.00533313]
 [ 0.01169561  0.00607555 -0.00331394 ...  0.00416719  0.0018944
  -0.00718195]
 ...
 [ 0.00564312 -0.00371559  0.00215051 ... -0.00369368  0.0037387
  -0.00432266]
 [ 0.01366517 -0.00443178 -0.00297376 ... -0.00379741  0.00209895
  -0.01051364]
 [ 0.00972784 -0.00346305 -0.00199818 ... -0.00662216  0.00317179
  -0.01270895]], shape=(100, 65), dtype=float32)


In [15]:
# predictoion at the first timestep
# it is 65 values representing prob of each character occuring enxt

time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 0.00955281 -0.00154437 -0.00473772 -0.00278807  0.00196686 -0.0009838
  0.00448267 -0.00170477 -0.00292637 -0.00286418 -0.00339941  0.00230169
 -0.00616335 -0.00087049  0.00276717 -0.00083601  0.00197117  0.00177544
  0.00516547 -0.00537197  0.00496557 -0.00050042 -0.00096579  0.00347758
 -0.00211366 -0.00114929  0.00601353 -0.00135903  0.01222187  0.00389455
 -0.0030307   0.0025818   0.00353166  0.00094268 -0.00245645 -0.00292408
 -0.00289008  0.00266008  0.00090725  0.00048019  0.00351178  0.00271758
  0.00397447 -0.00143788 -0.00235832  0.00104913 -0.01012344 -0.00738153
  0.00248223  0.00195208  0.00363751  0.00122833  0.00154249  0.00137983
  0.00249369  0.00025042  0.00032394  0.00429677  0.00437514 -0.00581795
  0.00250042  0.00246434 -0.00050634 -0.00050598 -0.00660942], shape=(65,), dtype=float32)


In [16]:
# need to make own loss function
# sample output distribution, will tell us the predicted characcter

sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
preicted_chars = int_to_text(sampled_indices)

predicted_chars = int_to_text(sampled_indices)

predicted_chars

" DZ? EHOn-bQNEjDlXH-ECbWkFZq\nA$c;r DLV3&gtb $l\nDz\nISyGWPzju&ish\nLme;&KtYWzte3'dM!kYRHwZ--.I'iQ-v&!P&"

In [17]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [18]:
# compile the model
model.compile(optimizer='adam',loss=loss)

In [19]:
# checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [21]:
# training
history = model.fit(data, epochs=1, callbacks=[checkpoint_callback])

172/172 [==============================] - 419s 2s/step - loss: 2.2653


In [23]:
#loading the model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [27]:
# generate the text

def generate_text(model, start_string):
    # num of chars to gen
    num_generate = 800
    
    # converting start string to numbers
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0) # turns a list into a nested list
    
    # empty string to store result
    generated_text = []
    
    # low temps results in more predictable text
    # higher results in more surprising text
    temperature = 1.0
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # remove batch dimension
        predictions = tf.squeeze(predictions, 0) # takes a nested list, removes the exterior list
        
        # using a categorical disstribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # we pass predicted char as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        
        generated_text.append(idx2char[predicted_id])
        
    return (start_string + ''.join(generated_text))
        

In [28]:
inp = input ("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeo
;NaZPRck$hlF'gk$
kj
zfzdCvc,b&sUymvY-q.Ih3MCPL
v:zR$GvkHKQ &N
PyakjCNE:rv'B
IZPhH$iV-V.rP.D?PQapD'3sdttIDy
Oh&VWA.GtED:eCROP?m?3DoL:'!-XHVgOXOmYzOapASwoccv
EAP!e W$r' ,XIv
.'Q!A
hpAKoi.'RjYca!LFQ-CLM.'DkJngvxyVLuQFcvbeA3Pkej.j!':OSev,'fgq3BUY!3:YG$Np'XDmhXa$TvmeTnFskk'd3'VJR;FVGq'!ip
&KFLYIC?ygU.&S$Utcqp$BqjVyMN'DEq?O
''ozxXuWx
Rtr'Eby$NiqK-c
Me-:!EJQ
BwePWQJgM:ohE'epsUUS:g&UtpEYcT,Bm
wrtqd&;,BdhwW
TYiJNBdMkyku,HU:YnyLP&gC.godi!OYsXVjiT-;i?LudmK'KMudZX-a;GFIi3rN&FWlxThvG'HVm: qqiLKUL lA Gf
,u,DbGCUpzc'meTTd,LyePtxdeN-k.XU,iCaI:F-CqBNJSxfGSBuGNR3XoBZJSsXEUikKL!vGvfiv$MWILNHB,jYF;XkXbFBQls?x';I;StG'PxzKzsZM;w3;tG?qzF?fDHhgl!o-;s$WiQheSFMrpYcWFDD'XQJ;eQP3zKYMrijy.ho!HDNEuIxa:uXN3T.3kpBOxftglMMuBAbYEWTvMVsF!qDULdDQaexJaHzb!dveej3BxncQ,$QUd3BN3rTzwuzIMi.MySwwf.E-wp;wrXpD'U.KBjzelnO$L.nBN
IDfUO
